# 1. First render

## Overview

This very first tutorial will guide you through an easy installation of the Mitsuba 2 framework and show you how to render you very first image!

<div class="alert alert-block alert-info">
    
✔️ **What will you learn?**
    
<ul>
  <li>(UPCOMING) Install mitsuba using pip install</li>
  <li>Load a scene from disk</li>
  <li>Render a scene</li>
  <li>Write a rendered image to disk</li>
</ul>
    
</div>

### Step 1 - install mitsuba and enoki

Before getting our hands dirty, we need to install the mitsuba and enoki libraries in our python environnement if it is not already done. While it is possible to build the framework from the source code (see [documentation](../src/getting_started/compilation.rst)), we recommand to use `pip` for an easier installation:

```
pip3 install mitsuba2
```

<div class="alert alert-block alert-danger">
⚠️ Unfortunately, it is not yet possible to install the mitsuba2 library using pip install. 
You will need to build the framework manually in the meantime. (see doc)
</div>

### Step 2 - choose your variant

Now that the framework was successfully installed, let's dive in and render our first image. As with all Python packages, the first thing to do is to import the libraries.

In [1]:
# Add mitsuba and enoki to PATH 
# (this shouldn't be necessary if you properly ran `source build/setpath.sh` before launching this notebook)
import sys
sys.path.append('../../../build/python')

In [1]:
import enoki as ek 
import mitsuba

We use the short alias `ek` for `enoki` as it will be used frequently in our scripts. Conviniently, the enoki API is very similar to the numpy API, and translating snippets of code from one framework to the other is often as easy as replacing the library suffix `np <--> ek`.

The organization of the Mitsuba framework is more involved, due to its correspondance to the underlying C++ library. Classes and functions are presented in three main submodules:

- `mitsuba.core`
- `mitsuba.render`
- `mitsuba.python`

Moreover, mitsuba is a *retargetable* system, meaning that it is possible to choose between different flavour of computational backends (e.g. CPU vs GPU), and change other aspects of the renderer (e.g. RGB vs spectral rendering, single vs double precision, etc...). Without going further into the details here, what's important to take away from this is that it is necessary to tell the system which *variant* of the renderer should be enabled before doing anything else.

This can be achieved using the `mitsuba.set_variant('variant_name')` routine. For more information on this subject, please refer to the [dedicated section](../src/getting_started/variants.rst) in the doc.

For this tutorial, we are going to stick with the simplest of the variant: `scalar_rgb`. As the name implies, computations will be performed in a scalar-fashion (e.g. non-vectorized code running on the CPU) and the light transport simulation will operate on RGB color values. As we will see in further tutorials, switching to a different variant of the system (e.g. GPU) should be possible by simply setting a different variant name here.

In [2]:
mitsuba.set_variant('scalar_rgb')

### Step 3 - loading a scene from disk

Now that the framework is properly setup, we can get started! 🚀

In this tutorial, we are going to load a Mitsuba scene from a file on disk. Mitsuba scenes are described using a simple and general XML-based format as explained in the [documentation](../src/getting_started/file_format.rst).

Luckily for us, a few scenes were already provided during the installation. In order to load a scene, we are going to use the [<code>mitsuba.core.load_file</code>][1] function to which we are going to pass the file name of our XML scene.


[1]: https://mitsuba2.readthedocs.io/en/latest/generated/core_api.html?highlight=load_file#mitsuba.core.load_file

In [3]:
from mitsuba.core import xml
scene = xml.load_file('../scenes/cbox.xml')

2021-11-18 09:34:20 INFO main [xml.cpp:1250] Loading XML file "..\scenes\cbox.xml" ..

2021-11-18 09:34:20 INFO main [xml.cpp:1251] Using variant "scalar_rgb"

2021-11-18 09:34:20 DEBUG main [PluginManager] Loading plugin "plugins\srgb.dll" ..

2021-11-18 09:34:20 DEBUG main [PluginManager] Loading plugin "plugins\srgb_d65.dll" ..

2021-11-18 09:34:20 DEBUG wrk0 [PluginManager] Loading plugin "plugins\path.dll" ..

...

2021-11-18 09:34:20 DEBUG wrk5 [OBJMesh] "cbox_greenwall.obj": computed vertex normals (took 0ms)

2021-11-18 09:34:20 DEBUG wrk2 [OBJMesh] Loading mesh from "cbox_luminaire.obj" ..

2021-11-18 09:34:20 DEBUG wrk2 [OBJMesh] "cbox_luminaire.obj": read 2 faces, 4 vertices (120 B in 0ms)

2021-11-18 09:34:20 DEBUG wrk2 [OBJMesh] "cbox_luminaire.obj": computed vertex normals (took 0ms)

2021-11-18 09:34:20 INFO main [Scene] Embree ready. (took 18ms)

### Step 4 - render the scene

With the scene loaded, we are now ready to perform our first rendering!

In [ ]:
image = scene.render(spp=128)

2021-11-18 09:34:20 INFO main [SamplingIntegrator] Starting render job (256x256, 128 samples, 8 threads)

### Step 5 - write image to file

This rendering routine returns a [<code>mitsuba.core.Bitmap</code>][mitsuba.core.Bitmap] object. It is then straightforward to write this image down to disk using the `write()` method.


[mitsuba.core.Bitmap]: https://mitsuba2.readthedocs.io/en/latest/generated/core_api.html?highlight=Bitmap#mitsuba.core.Bitmap

In [ ]:
image.write('my_first_rendering.exr')

2021-09-28 10:29:33 DEBUG main [Bitmap] Writing OpenEXR file "my_first_rendering.exr" (256x256, rgb, float16) ..

It is also possible to write a PNG image file, but for this we first need to convert our bitmap to make sure it has the right pixel and component format.

In [ ]:
from mitsuba.core import Bitmap, Struct
image = image.convert(Bitmap.PixelFormat.RGB, Struct.Type.UInt8, srgb_gamma=True)
image.write('my_first_rendering.png')

2021-09-28 10:29:34 DEBUG main [Bitmap] Writing PNG file "my_first_rendering.png" (256x256, rgb, uint8) ..

We can then easily display the rendered image in this notebook

In [ ]:
from IPython.display import Image
Image('my_first_rendering.png')

🎉 !! And BRAVO !! 🎉

You have now successfully installed Mitsuba on your system and performed your first render!

### *Reference API*

- [<code>mitsuba.set_variant</code>](https://mitsuba2.readthedocs.io/en/latest/generated/core_api.html#mitsuba.set_variant)
- [<code>mitsuba.core.load_file</code>](https://mitsuba2.readthedocs.io/en/latest/generated/core_api.html#mitsuba.core.load_file)
- [<code>mitsuba.python.util.render</code>](https://mitsuba2.readthedocs.io/en/latest/generated/python_api.html#mitsuba.python.util.render)
- [<code>mitsuba.core.Bitmap.write</code>](https://mitsuba2.readthedocs.io/en/latest/generated/core_api.html#mitsuba.core.Bitmap.write)